# search with lunr.js

In [1]:
import jinja2, bs4
with __import__("importnb").Notebook():
    from aggregate import get_frames, App

In [2]:
script =\
"""
<script type="module">
import lunr from "https://esm.sh/lunr"
import * as d3 from "https://cdn.jsdelivr.net/npm/d3@7/+esm"
import markdownIt from 'https://cdn.jsdelivr.net/npm/markdown-it@14.1.0/+esm'

var documents = {{df[["description"]].reset_index().dropna().to_json(orient="records")}};
var store = Object.fromEntries(documents.map(x => [x.index, x]))
var idx = lunr(function () {
  this.ref('index')
  this.field('description')
  documents.forEach(function (doc) {this.add(doc)}, this)
})
d3.select("#completion").selectAll("option").data(
    Object.keys(idx.invertedIndex)
).join("option").text(d => d)
function updateSearch(event) {
    event?.preventDefault()
    var table = d3.select(document.forms.search).select("table");
    var body = table.select("tbody");
    var ordering = "index description".split(" ");
    var tpl = document.forms.search.querySelector("table>template");
    
    var rows = body.selectAll("tr")
        .data(idx.search(document.forms.search.q.value))
      	.join("tr")
        .each((d, i, nodes)=>{
            console.log(d, i)
            var self = d3.select(nodes[i]);
            var entry = store[d.ref]
            self.selectAll("th").data([entry.index]).join("th").text(d => d)
            self.selectAll("td").data([entry.description]).join("td").html(d => markdownIt().render(d))
        })
       
        
}

var form = d3.select(document.forms.search)

document.forms.search.onsubmit = updateSearch
globalThis.lunr =  lunr
globalThis.d3 =  d3
globalThis.idx =  idx
globalThis.form =  form
globalThis.documents =  documents

updateSearch()
</script>
<form name=search>
    <label>query<input type=text name=q list=completion value=bottle></label>
    <input type=submit >
    <fieldset name=results>
        <legend>results</legend>
        <table>
            <thead>
                 <tr>
                     <th>project</th>
                     <th>description</th>
                 </tr>
            </thead>
            <tbody>
            </tbody>
            <template><tr><th></th><td></td></tr></template>
        </table>
    </fieldset>
    <datalist id=completion></datalist>
</form>
"""
file = """<html>
    <head></head>
    <body>{}</body>
</html>""".format(script)

In [5]:
@App.impl
def finalize(df):
    path = Path("lunr.html")
    path.write_text(
        html := jinja2.Environment().from_string(file).render(df=df)
    )
    print(path.absolute().as_uri())
    display(HTML(bs4.BeautifulSoup(html, features="html.parser").body))

In [ ]:
if __name__ == "__main__":
    app = App()
    app.manager.register(__import__(__name__))
    app.finalize()

In [ ]:
print()